### Major mysteries
* Why is the wall time inside the tasks so much higher for m1_4 and f4, but less so for m2_2/m4_1? What sort of cpu contention we have?
* What to measure so that we have a clear indicator for m2_2/m4_1 taking much longer than m1_4/f4? How to separate m1_4 and f4 cleanly?

### Other data (extracted manually from logs)
 - number of controller iterations (plan-act-await), number of events
   - f4 -> 198; 223
   - f1 -> 135; 135 (no transmits -> event count == iteration count == graph size)
   - m_4_1 -> 66; 135
   - m_2_2 -> 65; 165
   - m_1_4 -> 103; 232
   - there is much more event queuing in the multihost scenario, presumably because of the comm delay. Meaning the controller decides with less granularity
   - the event counts are underreported here because of a *bug* (remote transfer doesnt succ mark the input as present)
   - there is about the same number of data transmits in f4 and m1_4, meaning the final schedules are not that much different

### Next steps
 - (feature) replace remote transfers with local ones when it is possible
 - (bugfix) report remote transfer events correctly
 - (feature) extend tracing for controller phase, event batch size, phase duration, total runtime

In [1]:
import pandas as pd
import numpy as np

In [19]:
def fixWorker(df):
    rows = df['host'] != 'controller'
    df.loc[rows, 'worker'] = df.loc[rows, 'host'] + ':' + df.loc[rows, 'worker'] 

In [152]:
f1 = pd.read_json('lA_F_1.jsonl', lines=True)
f4 = pd.read_json('lA_F_4.jsonl', lines=True)
m1_4 = pd.read_json('lA_M_1_4.jsonl', lines=True)
fixWorker(m1_4)
m4_1 = pd.read_json('lA_M_4_1.jsonl', lines=True)
fixWorker(m4_1)
m2_2 = pd.read_json('lA_M_2_2.jsonl', lines=True)
fixWorker(m2_2)

In [283]:
def fixMode(df):
    rows = ~df.dataset.isna()
    proj = df[rows & ~df['mode'].isna()].set_index(['dataset', 'worker'])['mode']
    lookup = proj[~proj.index.duplicated(keep='last')]
    return df.set_index(['dataset', 'worker']).drop(columns='mode').join(lookup).reset_index()

def ensureColumns(df, columns):
    for column in columns:
        if not column in df.columns:
            df = df.assign(**{column: np.nan})
    return df
    
    
def transmitDurations(df):
    df = fixMode(df)
    datasets = df[~df.dataset.isna()].drop(columns='task')
    durations = datasets.pivot(index=['dataset', 'worker', 'mode'], columns=['action'], values=['at'])
    durations.columns = [name[1][len('transmit'):] for name in durations.columns]
    durations = durations.reset_index()
    localFix = durations['mode'] == 'local'
    durations.loc[localFix, 'Started'] = durations.loc[localFix, 'Finished']
    durations.loc[localFix, 'Loaded'] = durations.loc[localFix, 'Finished']
    durations = durations.assign(total=durations.Finished - durations.Planned)
    durations = durations.assign(commDelay=durations.Started-durations.Planned)
    durations = durations.assign(loadDelay=durations.Loaded-durations.Started)
    durations = durations.assign(transmitDelay=durations.Finished-durations.Loaded)
    return durations

def taskDurations(df):
    tasks = df[~df.task.isna()]
    durations = tasks.pivot(index=['task', 'worker'], columns=['action'], values=['at'])
    durations.columns = [name[1][len('task'):] for name in durations.columns]
    durations = durations.reset_index()
    durations = durations.assign(total=durations.Finished - durations.Planned)
    durations = durations.assign(commDelay = durations.Enqueued - durations.Planned)
    durations = durations.assign(queueDelay = durations.Started - durations.Enqueued)
    durations = durations.assign(runtimes = durations.Finished - durations.Started)
    durations = durations.assign(onWorker = durations.Finished - durations.Enqueued)
    return durations

def fmn(n):
    return f"{n:.3e}"

def analyzeTransmits(df):
    durations = transmitDurations(df)
    print(f"total transmit duration: {fmn(durations.total.sum())}")
    print(" *** ")
    print(f"mode counts: {durations['mode'].value_counts()}")
    print(f"per-mode transmit duration: {durations[['mode', 'total']].groupby('mode').sum()}")
    print(" *** ")
    print(f"total comm delay: {fmn(durations.commDelay.sum())}")
    print(f"mean comm delay: {fmn(durations.commDelay.mean())}")
    print(f"max comm delay: {fmn(durations.commDelay.max())}")
    print(" *** ")
    remotes = durations.query("mode == 'remote'")
    print(f"total load delay: {fmn(remotes.loadDelay.sum())}")
    print(f"mean load delay: {fmn(remotes.loadDelay.mean())}")
    print(f"max load delay: {fmn(remotes.loadDelay.max())}")
    print(" *** ")
    print(f"total transmit delay: {fmn(remotes.transmitDelay.sum())}")
    print(f"mean transmit delay: {fmn(remotes.transmitDelay.mean())}")
    print(f"max transmit delay: {fmn(remotes.transmitDelay.max())}")
    print(" *** ")
    
def analyzeTasks(df):
    durations = taskDurations(df)
    print(f"total task duration: {fmn(durations.total.sum())}")
    print(" *** ")
    print(f"total task duration per worker: {durations.groupby('worker').onWorker.agg(['mean', 'sum'])}")
    print(" *** ")
    print(f"total comm delay: {fmn(durations.commDelay.sum())}")
    print(f"mean comm delay: {fmn(durations.commDelay.mean())}")
    print(f"max comm delay: {fmn(durations.commDelay.max())}")
    print(" *** ")
    print(f"total queue delay: {fmn(durations.queueDelay.sum())}")
    print(f"mean queue delay: {fmn(durations.queueDelay.mean())}")
    print(f"max queue delay: {fmn(durations.queueDelay.max())}")
    print(" *** ")
    print(f"total runtime delay: {fmn(durations.runtimes.sum())}")

In [261]:
analyzeTasks(m2_2)

total task duration: 2.280e+10
 *** 
total task duration per worker:                     mean         sum
worker                              
h0:worker0  1.656845e+08  6130325490
h0:worker1  1.729176e+08  5533363920
h1:worker0  1.221324e+08  3663971949
h1:worker1  1.551724e+08  5586206020
 *** 
total comm delay: 1.890e+09
mean comm delay: 1.400e+07
max comm delay: 1.120e+09
 *** 
total queue delay: 1.674e+09
mean queue delay: 1.240e+07
max queue delay: 2.643e+07
 *** 
total runtime delay: 1.924e+10


In [262]:
analyzeTasks(m1_4)

total task duration: 2.848e+10
 *** 
total task duration per worker:                     mean         sum
worker                              
h0:worker0  1.930189e+08  6369623137
h0:worker1  1.880251e+08  5828778981
h0:worker2  1.549318e+08  5577544782
h0:worker3  2.129352e+08  7452732450
 *** 
total comm delay: 3.256e+09
mean comm delay: 2.412e+07
max comm delay: 1.030e+09
 *** 
total queue delay: 1.713e+09
mean queue delay: 1.269e+07
max queue delay: 3.221e+07
 *** 
total runtime delay: 2.352e+10


In [263]:
analyzeTasks(m4_1)

total task duration: 2.223e+10
 *** 
total task duration per worker:                     mean         sum
worker                              
h0:worker0  1.840998e+08  6627594351
h1:worker0  1.365749e+08  4506972624
h2:worker0  1.557204e+08  5138774422
h3:worker0  1.768042e+08  5834538220
 *** 
total comm delay: 1.208e+08
mean comm delay: 8.948e+05
max comm delay: 3.115e+06
 *** 
total queue delay: 1.741e+09
mean queue delay: 1.289e+07
max queue delay: 2.654e+07
 *** 
total runtime delay: 2.037e+10


In [264]:
task_f1 = taskDurations(f1)
analyzeTasks(f1)

total task duration: 1.783e+10
 *** 
total task duration per worker:                  mean          sum
worker                            
worker0  1.319854e+08  17818026891
 *** 
total comm delay: 1.052e+07
mean comm delay: 7.793e+04
max comm delay: 1.339e+05
 *** 
total queue delay: 1.369e+09
mean queue delay: 1.014e+07
max queue delay: 1.300e+07
 *** 
total runtime delay: 1.645e+10


In [265]:
task_f4 = taskDurations(f4)
analyzeTasks(f4)

total task duration: 2.809e+10
 *** 
total task duration per worker:                  mean         sum
worker                           
worker0  1.707579e+08  7001074310
worker1  2.555861e+08  6645239360
worker2  1.788079e+08  7331123557
worker3  2.630479e+08  7102292209
 *** 
total comm delay: 1.525e+07
mean comm delay: 1.130e+05
max comm delay: 3.712e+05
 *** 
total queue delay: 1.906e+09
mean queue delay: 1.412e+07
max queue delay: 2.605e+07
 *** 
total runtime delay: 2.617e+10


In [248]:
taskCompareF1F4 = task_f1.set_index(['task'])[['total']].rename(columns={'total': 'total1'}).join(task_f4.set_index(['task'])[['total']].rename(columns={'total': 'total4'}))
taskCompareF1F4 = taskCompareF1F4.assign(dif = taskCompareF1F4.total4 - taskCompareF1F4.total1)
taskCompareF1F4 = taskCompareF1F4.assign(rel = taskCompareF1F4.dif / taskCompareF1F4.total4)
taskCompareF1F4.sort_values(by="rel")

,total1,total4,dif,rel
task,,,,
concat:29989362f37d65adb916f9c7dde48777a53a1a352764051bd973beda2a63fb06,68975208,71734927,2759719,0.038471
concat:f26f97554dd3e073b8d9fe163f390f9f880154e2b75b9d7fb63a80b4690b6178,67527044,71302608,3775564,0.052951
"retrieve(0, 11):66952b78ebd28d018329479c6c5275924b0dc376168b2eb77226f5a3a1f9024c",70759868,76455586,5695718,0.074497
"retrieve(0, 20):8c8546823833230cde8b56a2e33825853ff3b2e06f49c666bbf44a3b4f7cc7b4",70741710,77079759,6338049,0.082227
"retrieve(0, 14):9094898c14e4cd09eeb51c92ffb36b4341530a0fdec4851c8c080ba668ba176a",73879619,80653899,6774280,0.083992
...,...,...,...,...
efi:3ca7e41b43cfc6dc0a5174c97a9da8ed0306a2046db7b34965a1728cbc107795,1046689611,2278063642,1231374031,0.540535
sot:7625257a28365dc17f52ac5bea8f6a86a99ce98b8c9f64c00c281ce5b1387a58,476056859,1062649327,586592468,0.552009
sot:904294dc9ee2f3ebabd25b17b13501b4c0e3261c488cae0cb266af2168bf33cd,471259792,1058127577,586867785,0.554629


In [251]:
taskCompareF1F4.sort_values(by="dif")[-10:]

,total1,total4,dif,rel
task,,,,
sot:87653e162e8d18cb86a8315791a60d4be2c884c28f93e65f597f77abd2e2e784,479118432,767614286,288495854,0.375834
sot:7183f05d8d693859986f8bbca7a14051459d4a6dcdc28078bf27d8f33cd1dc16,477998173,788788071,310789898,0.394009
sot:7625257a28365dc17f52ac5bea8f6a86a99ce98b8c9f64c00c281ce5b1387a58,476056859,1062649327,586592468,0.552009
sot:904294dc9ee2f3ebabd25b17b13501b4c0e3261c488cae0cb266af2168bf33cd,471259792,1058127577,586867785,0.554629
sot:8539458b812cacddc034b87113b2bb1014940ed0bfa652200571042c31e3ba92,470316374,1084441626,614125252,0.566305
sot:4ad92507a7167648b7fd517129af1b0f197056cd427f866c4eb0628d072d4f9a,478183254,1115544990,637361736,0.571346
efi:7da309b028ff75fc0ec9a12f75963566aa0d153c45ed6935c47c0e75563eee18,1045567051,2233195212,1187628161,0.531807
efi:ff11bcd117adc0690eacbc69dc108d176707b3617c984261d0e4c2d5327c00cb,1034777442,2231372911,1196595469,0.536260
efi:e7f579c85aef0f9112c4d5c7c42f0ff5ecaccf9d51f34661404a755714547cae,1050762145,2276816345,1226054200,0.538495


## Task Takeaways:
* There is a big difference between f1 and f4 in pure runtimes of tasks, 17e9 vs 28e9, suggesting some contention happening
  * Comparing individual tasks, we see only small relative/abs differences in concats and disk-accessing retrieves, but big in compute intensive sot or efi, suggesting there is some CPU contention
  * The difference is also visible for m scenarios -- m1_4 is expectedly like f4, but m2_2 and m4_1 are 20e9 being thus closer to f1. It could be that there is less overlap in those scenarios, as the scheduling is more gappy due to interleaved http comms?
* Queue delay exhibits no real difference over f/m scenarios
* Comm delays are 1e7 for f scenarios, 1e8 for m4_1, and 1e9 for m2_2 and m1_4 -- m4_1 being midway looks more like a glitch
* m2_2 is showing a slight disbalance of one worker being less utilised than the others, all others look balanced

In [284]:
analyzeTransmits(f4)

total transmit duration: 8.337e+06
 *** 
mode counts: mode
local    88
Name: count, dtype: int64
per-mode transmit duration:          total
mode          
local  8336637
 *** 
total comm delay: 8.337e+06
mean comm delay: 9.473e+04
max comm delay: 2.603e+05
 *** 
total load delay: 0.000e+00
mean load delay: nan
max load delay: nan
 *** 
total transmit delay: 0.000e+00
mean transmit delay: nan
max transmit delay: nan
 *** 


In [285]:
trans_m2_2 = transmitDurations(m2_2)
analyzeTransmits(m2_2)

total transmit duration: 6.179e+09
 *** 
mode counts: mode
remote       50
local        27
redundant     8
Name: count, dtype: int64
per-mode transmit duration:                   total
mode                   
local      2.513151e+08
redundant  1.774257e+09
remote     4.153092e+09
 *** 
total comm delay: 1.853e+09
mean comm delay: 2.180e+07
max comm delay: 6.395e+08
 *** 
total load delay: 2.685e+07
mean load delay: 5.370e+05
max load delay: 1.393e+06
 *** 
total transmit delay: 3.289e+09
mean transmit delay: 6.577e+07
max transmit delay: 1.321e+09
 *** 


In [289]:
fmn(trans_m2_2.query("mode == 'redundant'").total.sum())

'1.774e+09'

In [287]:
analyzeTransmits(m4_1)

total transmit duration: 8.200e+09
 *** 
mode counts: mode
remote    99
Name: count, dtype: int64
per-mode transmit duration:              total
mode              
remote  8200233417
 *** 
total comm delay: 1.800e+09
mean comm delay: 1.818e+07
max comm delay: 6.994e+08
 *** 
total load delay: 4.834e+07
mean load delay: 4.882e+05
max load delay: 1.260e+06
 *** 
total transmit delay: 6.352e+09
mean transmit delay: 6.416e+07
max transmit delay: 1.532e+09
 *** 


In [288]:
trans_m1_4 = transmitDurations(m1_4)
analyzeTransmits(m1_4)

total transmit duration: 8.925e+08
 *** 
mode counts: mode
local    97
Name: count, dtype: int64
per-mode transmit duration:            total
mode            
local  892488342
 *** 
total comm delay: 8.925e+08
mean comm delay: 9.201e+06
max comm delay: 4.570e+08
 *** 
total load delay: 0.000e+00
mean load delay: nan
max load delay: nan
 *** 
total transmit delay: 0.000e+00
mean transmit delay: nan
max transmit delay: nan
 *** 


## Transmit Takeaways
* The number of redundant transfers is low, just 8, in the 2-2 scenario. However, they still contributed 1e9 to the total runtime!
* Much more remote than local transfers in the 2-2 scenario -- 166 vs 27
* Mean comm delay for m1_4 is 9e6 whereas for m4_1 its 1e7 -- suggesting number of hosts is not that important on this front

In [274]:
print(trans_m2_2[["dataset"]].value_counts().sum())
trans_m2_2[["dataset"]].value_counts()

85


dataset                                                                         
concat:29989362f37d65adb916f9c7dde48777a53a1a352764051bd973beda2a63fb06             3
retrieve(1, 6):3e415bf5fc9cbf182572a5f1a02bd3020172b2f6bf4dd6bf7c04b6384fa620d5     3
retrieve(0, 16):b0f29339bf6aab0a4f71dbd2b43be96d1583726c2f7606cebc4263aaa3b406ae    3
retrieve(1, 2):c2e4cafac05f7f16f023ed076d7f004e03b111f725e2447d654aa1f1c94685d1     3
retrieve(0, 18):79caabb3af47c5c5d9f1d340b1577ea2d1450aab543213fb2db99b1df980f88b    2
                                                                                   ..
retrieve(1, 12):419cab96aa86998d59dfff78a4d9acaf053fbb3dc7b4007fc550ff6a6d83bd22    1
retrieve(1, 15):95098679dce6c80d62d074a9bb161d1e2d4b0acb16ae2bf717e093d6cdd39236    1
retrieve(1, 16):a3ae8851be39a9dce16d214b1be60f2c1d9b3adde75674001763570a6ddd1cb7    1
retrieve(1, 17):58eb1753fa0fec3ea14f09bfc2f9ee15b03b32ebe652798cd2f4844adf11e9db    1
threshold:df78e83f1ba4dd978d013474193ef558ddfe940e84572973f

In [275]:
print(trans_m1_4[["dataset"]].value_counts().sum())
trans_m1_4[["dataset"]].value_counts()

97


dataset                                                                         
concat:29989362f37d65adb916f9c7dde48777a53a1a352764051bd973beda2a63fb06             3
concat:f26f97554dd3e073b8d9fe163f390f9f880154e2b75b9d7fb63a80b4690b6178             3
retrieve:228aecd8e8e69130f9f6fe68973639b52a6e378ce14e7372989762a0d4ca9118           3
retrieve(0, 16):b0f29339bf6aab0a4f71dbd2b43be96d1583726c2f7606cebc4263aaa3b406ae    3
retrieve(0, 6):06ff000f29b3ec409ecbf4826ef59f88e20ae984da898cf200a756c9fa448e1f     2
                                                                                   ..
retrieve(1, 20):ddb2a10425126ae3147cc5d38d478a6b97daa2b3d426e21e7d1ec1785505bb18    1
retrieve(1, 3):b4bf80f8cbd0d9bd0c8b8bd183835151ccdbac05ab2bd03b7260531c7cfe3211     1
retrieve(0, 1):b544ed58bf8a08a4b5c630cfe77040f0d79f00ec1b9c1e9655b737d1bf842791     1
retrieve(1, 6):3e415bf5fc9cbf182572a5f1a02bd3020172b2f6bf4dd6bf7c04b6384fa620d5     1
threshold:df78e83f1ba4dd978d013474193ef558ddfe940e84572973f